In [ ]:
from google.colab import drive  
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/Kuliah Teknik Informatika/Semester 5C/CAPSTONE PROJEK/CHATBOT'

In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

In [ ]:
!pip install nltk

In [ ]:
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

In [ ]:
#Membuat Variabel
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open("/content/drive/MyDrive/Kuliah Teknik Informatika/Semester 5C/CAPSTONE PROJEK/CHATBOT/intents.json").read()
intents = json.loads(data_file)

In [ ]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # mentokenisasi setiap kata
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # menambahkan documents kedalam corpus
        documents.append((w, intent["tag"]))

        # menambahkan ke list classes kita
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [ ]:
# lemmatize pada setiap kata
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)

#menyimpan olahan kata kedalam bentuk model
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [ ]:
# membuat training data dan array kosong untuk output
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenisasi untuk setiap pola
    pattern_words = doc[0]
    # lemmatize setiap kata - create kata dasar, agar menjadi key bagi bentuk kata ubahnya
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, jika keccocokan kata di temukan dalam pola saat ini
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for setiap tag and '1' for tag saat ini(untuk setiap pola)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# acak semua features and ubah kedalam bentuk np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

In [ ]:
# implemntasi sequintial pada data train
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

In [ ]:
# algoritma untuk menentukan nilai akurasi
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [ ]:

 from keras import callbacks
 earlystopping = callbacks.EarlyStopping(monitor ="loss", mode ="min", patience = 5, restore_best_weights = True)
callbacks =[earlystopping]

In [ ]:
 # fitting dan simpan model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5", hist)
print("model created")

In [ ]:
# membersikan kata dari simbol yang tidak diperlukan
def clean_up_sentence(sentence):
  # tokenisasi pola-memisahkan kata kedalam bentuk array
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# membuat function kamus bag of words. array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize pola
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # isi 1 if jika kata saat ini ada di posisi kosakata
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

In [ ]:
# membuat function prediksi jawaban
def predict_class(sentence, model):
    # menyaring prediksi dibawah threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # mengurutkan berdasarkan probabilitas
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# membuat function untuk memberi respon
def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [ ]:
# menampilkan respon berdasarkan prediksi
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    response = print(res)
    return response

In [ ]:
while True:
    input_data = input("You Message : ")
    if input_data == ('exit'):
        print("Terima kasih")
        break
    answer = chatbot_response(input_data)
    print("Bot answer : ", answer)